# State class
틱택토 상태를 나타낼 수 있는 class를 만들어 주세요.

- 2명의 플레이어가 있고, 각 플레이어의 수가 뭔지 식별가능해야 합니다.  

**꼭 구현해야할 매소드(주니어/시니어 공통)**
- 둘 수 있는 위치가 어딘지 알려주는 함수
- win / lose / draw를 식별해주는 함수
- render 함수

**min-max algorithm / alpha-beta algorithm 구현(시니어)**

### 🚨 만약 감이 안 온다면 운영진에게 연락주시면 참고할 만한 레퍼런스를 보내드리겠습니다.
### 🚨 참고한 레퍼런스가 있다면, 출처를 남겨주세요.

틱택토 State

\<레퍼런스>

[rl_practice_2_5.ipynb](https://colab.research.google.com/drive/1pMYjlOL2JUjlOovl6mlEkipf4SQOjkcO?usp=sharing)

# Code

In [ ]:
import numpy as np

In [ ]:
GOING_FIRST = 1
GOING_SECOND = -1
EMPTY_TILE = 0

RENDERING_SYMBOL = {GOING_FIRST : 'O', GOING_SECOND : 'X', EMPTY_TILE : '-'}

BOARD_ROWS = 3
BOARD_COLS = 3

# POS_TO_ACTIDX = {(0,0) : 0, (0,1) : 1, (0,2) : 2,
#                  (1,0) : 3, (1,1) : 4, (1,2) : 5,
#                  (2,0) : 6, (2,1) : 7, (2,2) : 8}
# but 3x3 게임판이 아닐 경우 ... 다시 써야함

In [ ]:
'''
Position : Action Index

(0,0) : 0, (0,1) : 1, (0,2) : 2,
(1,0) : 3, (1,1) : 4, (1,2) : 5,
(2,0) : 6, (2,1) : 7, (2,2) : 8
'''
def pos_to_actidx(row_idx, col_idx):
    action_idx = 99
    if 0 <= row_idx < BOARD_ROWS and 0 <= col_idx < BOARD_COLS:
        action_idx = 3*row_idx + col_idx
    return action_idx

In [ ]:
class State:
    def __init__(self, board_rows=BOARD_ROWS, board_cols=BOARD_COLS):
        self.board_rows = board_rows
        self.board_cols = board_cols
        self.board_size = board_rows * board_cols

        self.board = np.zeros(shape=[board_rows, board_cols], dtype=int) # 3X3 게임판 초기 상태

        # self.pos_to_actidx = POS_TO_ACTIDX

        self.empty_tile = EMPTY_TILE # 0
        self.going_first = GOING_FIRST # 1
        self.going_second = GOING_SECOND # -1

        self.rendering_symbol = RENDERING_SYMBOL

        self.winner = None
        self.end = None

    # 가능한 액션 인덱스를 반환하는 함수 (수를 둘 수 있는 위치가 어딘지 알려주는 함수)
    def get_available_actions(self):
        if self.is_end_state():
            available_positions = []
        else:
            available_positions = [
                (i, j) for i in range(self.board_rows)
                       for j in range(self.board_cols) if self.board[i, j] == self.empty_tile
            ]

        available_action_ids = []
        for pos in available_positions:
            # available_action_ids.append(self.pos_to_actidx[pos])
            available_action_ids.append(pos_to_actidx(pos[0], pos[1]))

        return available_action_ids

    # 최종 게임 결과 식별 함수 (누가 이겼는지 혹은 무승부인지)
    def is_end_state(self):
        if self.end is not None:

            return self.end

        results = []

        # 게임판 가로 1줄씩 승리조건 확인
        for i in range(self.board_rows):
            results.append(np.sum(self.board[i, :]))

        # 게임판 세로 1줄씩 승리조건 확인
        for i in range(self.board_cols):
            results.append(np.sum(self.board[:, i]))

        # 게임판 대각선 승리조건 확인
        trace = 0
        reverse_trace = 0

        for i in range(self.board_rows):
            trace += self.board[i, i] # 왼쪽 위 ~ 오른쪽 아래 대각선
            reverse_trace += self.board[i, self.board_rows - 1 - i] # 오른쪽 위 ~ 왼쪽 아래 대각선

        results.append(trace)
        results.append(reverse_trace)

        # GOING_FIRST(선공) 또는 GOING_SECOND(후공) 승리 조건 확인
        # (한 줄의 합이  3이면 ->  1+1+1 선공 승리)
        # (한 줄의 합이 -3이면 -> -1-1-1 후공 승리)
        for result in results:
            if result == 3 or result == -3:
                self.end = True
                if result == 3:
                    self.winner = self.going_first
                else:
                    self.winner = self.going_second

                return self.end

        # 무승부 확인
        # (9칸 전부 수를 둔 경우 -> 승부가 나지 않음)
        sum_values = np.sum(np.abs(self.board))
        if sum_values == self.board_size:
            self.winner = 0
            self.end = True

            return self.end

        # 게임이 아직 끝나지 않음
        self.end = False

        return self.end

    # render 함수
    def get_state_as_board(self):
        rendering_board = ''
        for i in range(self.board_rows):
            out = ''
            for j in range(self.board_cols):
                out += ' ' + self.rendering_symbol[int(self.board[i, j])] + ' ' # (board[i,j] 특정 좌표에 착수할 때 1(선공) 또는 -1(후공)으로 표시)
            rendering_board += out + '\n'

        return rendering_board

In [ ]:
state = State()

state.board = np.array([[0, -1, 1],
                        [1, -1, 1],
                        [-1, 0, 1]])

available_act_ids = state.get_available_actions()
end = state.is_end_state()
render = state.get_state_as_board()

print(available_act_ids)
print()
print(render)
print(f'Game Over : {end}')
print(f'Winner : {state.winner}')

[]

 -  X  O 
 O  X  O 
 X  -  O 

Game Over : True
Winner : 1


In [ ]:
state = State()

state.board = np.array([[0, -1, 1],
                        [1, -1, 1],
                        [-1, 0, 0]])

available_act_ids = state.get_available_actions()
end = state.is_end_state()
render = state.get_state_as_board()

print(available_act_ids)
print()
print(render)
print(f'Game Over : {end}')
print(f'Winner : {state.winner}')

[0, 7, 8]

 -  X  O 
 O  X  O 
 X  -  - 

Game Over : False
Winner : None


# Code

Min-max algorithm / Alpha-beta algorithm

Zero-sum game

: 어떤 한 플레이어에게 좋은 건 반대로 다른 플레이어에게는 나쁜 게임을 의미한다. 자신이 점수를 얻으면 상대방은 결국 점수를 잃는 것이고, 두 사람의 결과를 더해보면 0이 되는 것이다.

***

Min-max algorithm

: 턴제 게임에서 상대방이 최선의 수를 택할 것이라고 가정하고 다음 수를 예측한다. **상대방의 최선의 수는 나의 입장에서는 최악의 수**이기 때문에 '상대방의 최선의 수를 두더라도 이에 대응하여 최소한의 손실을 보는 것'이 궁극적인 목표라 할 수 있다.

(선공자 입장에서의 알고리즘) 선공: 내 최선의 수(max) → 후공: 상대방 최선의 수(내 기준 최악의 수)(min) → ...

상대방인 MIN 플레이어가 나를 지게 만들기 위해 최선의 선택을 하는 worst-case에서 MAX 플레이어인 내 결과를 최대화하는 행동을 택한다.

두 플레이어(최대화 플레이어와 최소화 플레이어)가 번갈아 가며 최적의 수를 찾기 위해 사용됩니다. 각 단계에서:

최대화 플레이어는 자신의 점수를 최대화하려고 하고,
최소화 플레이어는 상대방(최대화 플레이어)의 점수를 최소화하려고 합니다.

**단말 노드의 평가 함숫값이 높을수록 MAX가 유리하고, 낮을수록 MIN이 유리하다.**

***

Alpha-beta pruning

: 최선의 수를 두는 플레이어의 선택에 최종적으로 영향을 끼치지 않는 가지를 삭제

Alpha : 현재까지 내가 이기기 위해 찾은 최고 점수

Beta : 현재까지 상대방이 나를 지게 만들기 위해 찾은 최저 점수

***

\<Tic-Tac-Toe 규칙>

- 선공, 후공 차례대로 번갈아가면서 수를 둔다.

- Max가 선공. (선공 : 'O', +1)

- 같은 무늬의 직선 한 줄(가로, 세로, 대각)을 먼저 완성하는 쪽이 승리.

제로썸 게임이다. (승자와 패자가 나뉜다.)

MinMax 알고리즘을 사용하기에 플레이어들은 최선의 수를 선택한다는 가정하에 게임을 진행한다.

\<레퍼런스>

[위키백과 - 알파-베타 가지치기](https://ko.wikipedia.org/wiki/%EC%95%8C%ED%8C%8C-%EB%B2%A0%ED%83%80_%EA%B0%80%EC%A7%80%EC%B9%98%EA%B8%B0#cite_note-RN10-1)

[[인공지능 기초] Adversarial Search - Minimax Search와 Alpha-beta Pruning](https://glanceyes.com/entry/Adversarial-Search%EC%9D%98-Minimax-Search%EC%99%80-Alpha-beta-Pruning)

[MinMax알고리즘을 이용한 TicTacToe](https://velog.io/@h_ani99/MinMax%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-TicTacToe)

[알파-베타 가지치기, Alpha-beta pruning](https://going-to-end.tistory.com/entry/%EC%95%8C%ED%8C%8C-%EB%B2%A0%ED%83%80-%EA%B0%80%EC%A7%80%EC%B9%98%EA%B8%B0-Alpha-beta-pruning)

In [ ]:
def minimax(state, depth, maxPlayer):
    pos = (-1, -1)

    # Terminal node이면 보드를 평가해 위치와 평가값을 반환
    if depth == 0 or state.is_end_state():
        return pos, evaluate(state)

    if maxPlayer:
        value = -10000  # 음의 무한대
        for p in state.get_available_actions(): # p : action index
            state.board[p // state.board_cols, p % state.board_cols] = state.going_first # MAX가 p의 행동을 했다고 가정할 때 (해당 좌표에 착수했다고 가정할 때)
            x, score = minimax(state, depth-1, False) # 다음 턴; 후공 MIN 차례 (MAX는 MIN이 자기 기준 최솟값을 선택할 것으로 생각함)
            state.board[p // state.board_cols, p % state.board_cols] = state.empty_tile # 시뮬레이션 해보고 원래대로 되돌려두기
            if score > value: # score가 현재 가지고 있는 value보다 크면
                value = score  # 최댓값을 취한다.
                pos = p  # 최댓값의 위치 좌표 튜플을 기억한다.
    else:
        value = 10000  # 양의 무한대
        for p in state.get_available_actions():
            state.board[p // state.board_cols, p % state.board_cols] = state.going_second
            x, score = minimax(state, depth-1, True)
            state.board[p // state.board_cols, p % state.board_cols] = state.empty_tile
            if score < value:
                value = score  # 최솟값을 취한다.
                pos = p  # 최솟값의 위치 좌표 튜플을 기억한다.

    return pos, value

\<Min-max algorithm>

MAX 플레이어는 가능한 모든 행동(착수 위치)을 시뮬레이션하면서, 그 행동을 할 경우 MIN 플레이어가 취할 수 있는 최솟값을 추정합니다. 이는 `minimax(state, depth-1, False)`로 구현됩니다. 즉, **MAX는 MIN이 자신에게 최악의 경우를 만들려고 할 때, 그 결과를 분석**하는 겁니다.

score > value 조건에서, MAX 플레이어는 항상 더 높은 점수를 선택하려고 합니다. 이때 value는 MAX가 고려 중인 최댓값입니다. **score는 MIN의 수를 고려한 후의 예상 점수인데, 이 값이 현재 MAX가 갖고 있는 최댓값보다 크면, 더 나은 결과이므로 그 값을 업데이트합니다.**

(깊이 우선 검색의 최적화: 깊이 제한을 더 지능적으로 설정할 수도 있습니다. 예를 들어, 어떤 수는 더 깊게 탐색할 가치가 있을 때 더 깊이 탐색하거나, 특정 수는 더 얕은 깊이에서만 평가해도 될 수 있습니다. 이와 같은 전략적 깊이 설정을 통해 탐색 효율성을 높일 수 있습니다.

평가 함수 개선: 평가 함수는 미니맥스 알고리즘의 성능과 정확성에 큰 영향을 미칩니다. 기본적인 평가 함수 외에, 게임의 특성에 맞는 정교한 평가 함수를 설계하면 더 나은 플레이 성능을 기대할 수 있습니다. 특히, 중간 게임 단계에서 플레이어의 승리 가능성을 더 잘 반영하도록 조정할 수 있습니다.)

In [ ]:
def alphabeta(state, depth, a, b, maxPlayer): # a (alpha) : MAX 플레이어가 찾은 최댓값 / b (beta) : MIN 플레이어가 찾은 최솟값
    pos = (-1, -1)

    # 터미널 노드이거나 depth가 0에 도달하면 평가값 반환
    if depth == 0 or state.is_end_state():
        return pos, evaluate(state)

    if maxPlayer:
        value = -10000
        for p in state.get_available_actions():
            state.board[p // state.board_cols, p % state.board_cols] = state.going_first
            x, score = alphabeta(state, depth - 1, a, b, False)
            state.board[p // state.board_cols, p % state.board_cols] = state.empty_tile

            if score > value:
                value = score  # 최댓값 업데이트
                pos = p  # 최적의 위치 업데이트
            a = max(a, value) # 확보한 최댓값인 alpha와 현재 가진 value 비교하여 더 큰 값을 alpha로 한다.
            if a >= b: # 만약 MAX 플레이어가 확보한 alpha 값이 beta 값보다 크다면, MIN 플레이어가 이미 더 작은 값을 선택할 가능성이 있기 때문에 더 큰 값은 고려할 필요가 없음
                break  # b 컷: 부모 노드보다 크면 더 탐색할 필요 없음

    else:
        value = 10000
        for p in state.get_available_actions():
            state.board[p // state.board_cols, p % state.board_cols] = state.going_second
            x, score = alphabeta(state, depth - 1, a, b, True)
            state.board[p // state.board_cols, p % state.board_cols] = state.empty_tile

            if score < value:
                value = score  # 최솟값 업데이트
                pos = p  # 최적의 위치 업데이트
            b = min(b, value)
            if a >= b:
                break  # a 컷: 부모 노드보다 작으면 더 탐색할 필요 없음

    return pos, value

\<Alpha-beta pruning>

알파베타 가지치기의 핵심은 각 플레이어가 자신에게 유리한 선택을 할 때, 상대가 더 나은 선택을 방해할 수 있는지 미리 확인하여 탐색을 줄이는 것!!

※ α >= β일 때 가지치기가 발생하는 이유

- 알파(α)와 베타(β)의 의미

    α (알파): MAX 플레이어가 현재까지 찾은 최고의 점수. MAX는 가능한 한 이 알파보다 높은 점수를 얻으려고 한다.

    β (베타): MIN 플레이어가 현재까지 찾은 최악의 점수. MIN은 가능한 한 이 베타보다 낮은 점수를 얻으려고 한다.
    ***

- MAX와 MIN의 전략 차이

    MAX 플레이어는 가장 높은 점수를 얻으려고 하고,

    MIN 플레이어는 가장 낮은 점수를 얻으려고 한다.
    ***

- 핵심 아이디어: 왜 α >= β일 때 가지치기할까?

    α >= β라는 상태는, MAX와 MIN의 목표가 충돌하여 더 이상 탐색할 필요가 없다는 걸 의미함

    → 단계별 이해

    1. 현재까지 확인된 알파 값 α:

    MAX는 현재까지 탐색한 노드들에서 자신에게 유리한 값을 찾아가는 중. 이때 최고로 유리한 값이 알파(α)이다! 즉, **MAX는 이 알파보다 낮은 점수는 선택하지 않을 것**.

    2. 현재까지 확인된 베타 값 β:

    MIN은 상대방(MAX)이 탐색하는 도중, 자신이 막을 수 있는 최선의 방법을 찾고 있습니다. MIN이 찾은 가장 좋은 선택(**자신에게 최악인 점수 중에 최소의 값**)이 베타(β). **MIN은 이 베타보다 높은 점수는 주지 않으려고 할 것이다.**

    3. 알파, 베타의 의미가 충돌하는 순간 (α >= β):

    MAX는 현재 α 이상의 점수를 이미 확보할 수 있는 상황. 즉, **MAX는 α보다 낮은 점수는 고려하지 않을 것이고, 그 이상을 노리려고 함.**

    MIN은 현재 β 이하의 점수로 막으려고 하고 있다. 그런데 이 상황에서 α가 β보다 크거나 같은 경우, MIN은 더 낮은 점수를 선택할 필요가 없다.**어차피 MAX는 MIN의 베타 값보다도 더 큰 알파 값 이상의 값을 확보했기 때문에!**